# Collision Avoidance - Live Demo

In this notebook we'll use the model we trained to detect whether the robot is ``free`` or ``blocked`` to enable a collision avoidance behavior on the robot.  

## Load the trained model

We'll assumed that you've already downloaded the ``best_model.pth`` to your workstation as instructed in the training notebook.  Now, you should upload this model into this notebook's
directory by using the Jupyter Lab upload tool.  Once that's finished there should be a file named ``best_model.pth`` in this notebook's directory.  

> Please make sure the file has uploaded fully before calling the next cell

Execute the code below to initialize the PyTorch model.  This should look very familiar from the training notebook.

In [ ]:
import torch
import torchvision
import torch.nn.functional as F
import cv2
import numpy as np
import random
import time
import traitlets
from IPython.display import display, Audio
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg, Robot


#
# Robot and Audio



# Start update loop


In [ ]:
#Load Blocked/Free Model
model_blocked = torchvision.models.alexnet(pretrained=False)
model_blocked.classifier[6] = torch.nn.Linear(model_blocked.classifier[6].in_features, 2)  # 2 classes: Free, Blocked
model_blocked.load_state_dict(torch.load('best_model.pth'))

# Load Collision Type Model (shoes vs others)
model_type = torchvision.models.alexnet(pretrained=False)
model_type.classifier[6] = torch.nn.Linear(model_type.classifier[6].in_features, 2)  # 2 classes: Shoes, Others
model_type.load_state_dict(torch.load('best_model2.pth'))


In [ ]:
#
device = torch.device('cuda')
model_blocked = model_blocked.to(device)
model_type = model_type.to(device)

# Preprocessing
mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])
normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    x = cv2.cvtColor(camera_value, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x


In [ ]:
 #Widgets
camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')
collision_type_label = widgets.Label(value="Collision Type: None")
speed_slider = widgets.FloatSlider(description='speed', min=0.0, max=0.5, value=0.0, step=0.01, orientation='horizontal')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
display(widgets.VBox([widgets.HBox([image, blocked_slider]), collision_type_label, speed_slider]))


In [ ]:
robot = Robot()


In [ ]:
def update(change):
    global last_audio_time

    x = preprocess(change['new'])

    # Run blocked/free detection
    y_blocked = model_blocked(x)
    probs_blocked = F.softmax(y_blocked, dim=1).flatten().detach().cpu().numpy()
    prob_blocked = float(probs_blocked[0])  # class 0: blocked, class 1: free
    blocked_slider.value = prob_blocked

    if prob_blocked < 0.5:
        # Free path
        collision_type_label.value = "Collision Type: None"
        robot.forward(speed_slider.value)
    else:
        # Blocked path -> Run second model to detect type
        y_type = model_type(x)
        probs_type = F.softmax(y_type, dim=1).flatten().detach().cpu().numpy()
        prob_shoes, prob_others = probs_type

        if prob_shoes > prob_others:
            collision_type = "Shoes"
            print(collision_type)
            
        else:
            collision_type = "Other"
            print(collision_type)
            

        collision_type_label.value = f"Collision Type: {collision_type}"
        robot.left(speed_slider.value)

        

In [ ]:
# Blocked path -> Run second model to detect type
y_type = model_type(x)
probs_type = F.softmax(y_type, dim=1).flatten().detach().cpu().numpy()
prob_shoes, prob_others = probs_type

# Confidence check
if max(prob_shoes, prob_others) > CONFIDENCE_THRESHOLD:
    if prob_shoes > prob_others:
        prediction_buffer.append("Shoes")
    else:
        prediction_buffer.append("Other")
else:
    prediction_buffer.append("Uncertain")

# Determine most common prediction
if prediction_buffer:
    most_common = collections.Counter(prediction_buffer).most_common(1)[0][0]
else:
    most_common = "Uncertain"

if most_common != "Uncertain":
    collision_type_label.value = f"Collision Type: {most_common}"
    print("Detected:", most_common)
else:
    collision_type_label.value = "Collision Type: Uncertain"
    print("Detected: Uncertain")

# ▶️ Random action instead of fixed left turn
action = random.choice(["left", "right", "backward"])
print("Avoidance Action:", action)

if action == "left":
    robot.left(speed_slider.value)
elif action == "right":
    robot.right(speed_slider.value)
else:
    robot.backward(speed_slider.value)

time.sleep(0.5)  # allow the action to finish



In [ ]:
update({'new': camera.value})
camera.observe(update, names='value')


In [ ]:
robot.stop()

In [ ]:
###############################old code  ###########################################################################

In [1]:
import torch
import torchvision

model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

In [2]:
from IPython.display import Audio, display

Next, load the trained weights from the ``best_model.pth`` file that you uploaded

In [ ]:
model.load_state_dict(torch.load('best_model.pth'))

Currently, the model weights are located on the CPU memory execute the code below to transfer to the GPU device.

In [ ]:
device = torch.device('cuda')
model = model.to(device)

### Create the preprocessing function

We have now loaded our model, but there's a slight issue.  The format that we trained our model doesn't *exactly* match the format of the camera.  To do that, 
we need to do some *preprocessing*.  This involves the following steps

1. Convert from BGR to RGB
2. Convert from HWC layout to CHW layout
3. Normalize using same parameters as we did during training (our camera provides values in [0, 255] range and training loaded images in [0, 1] range so we need to scale by 255.0
4. Transfer the data from CPU memory to GPU memory
5. Add a batch dimension

In [47]:
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

Great! We've now defined our pre-processing function which can convert images from the camera format to the neural network input format.

Now, let's start and display our camera.  You should be pretty familiar with this by now.  We'll also create a slider that will display the
probability that the robot is blocked.  We'll also display a slider that allows us to control the robot's base speed.

In [48]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')
speed_slider = widgets.FloatSlider(description='speed', min=0.0, max=0.5, value=0.0, step=0.01, orientation='horizontal')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.VBox([widgets.HBox([image, blocked_slider]), speed_slider]))

We'll also create our robot instance which we'll need to drive the motors.

In [49]:
from jetbot import Robot

robot = Robot()

Next, we'll create a function that will get called whenever the camera's value changes.  This function will do the following steps

1. Pre-process the camera image
2. Execute the neural network
3. While the neural network output indicates we're blocked, we'll turn left, otherwise we go forward.

In [25]:
# import torch.nn.functional as F
# import time

# def update(change):
#     global blocked_slider, robot
#     x = change['new'] 
#     x = preprocess(x)
#     y = model(x)
    
#     # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
#     y = F.softmax(y, dim=1)
    
#     prob_blocked = float(y.flatten()[0])
    
#     blocked_slider.value = prob_blocked
    
#     if prob_blocked < 0.5:
#         robot.forward(speed_slider.value)
#     else:
#         display(Audio("audioFiles/audio.wav", autoplay=True)) 
#         robot.left(speed_slider.value)
    
#     time.sleep(0.001)
        
# update({'new': camera.value})  # we call the function once to initialize

In [50]:
from IPython.display import Audio, display
import time
import torch.nn.functional as F

# Initialize this variable outside the function (e.g. at the top of the notebook)
last_audio_time = 0
AUDIO_COOLDOWN = 3.0  # seconds

def update(change):
    global blocked_slider, robot, last_audio_time
    
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
    y = F.softmax(y, dim=1)
    prob_blocked = float(y.flatten()[0])
    
    blocked_slider.value = prob_blocked
    
    if prob_blocked < 0.5:
        robot.forward(speed_slider.value)
    else:
        current_time = time.time()
        if current_time - last_audio_time > AUDIO_COOLDOWN:
            display(Audio("audioFiles/audio3.wav", autoplay=True))
            last_audio_time = current_time
        robot.left(speed_slider.value)
    
    time.sleep(0.001)

# Initialize once
update({'new': camera.value})

In [8]:
# from IPython.display import Audio, display
# import time
# import torch.nn.functional as F
# import random
# AUDIO_FILES = [
#     "audioFiles/audio.wav",
#     "audioFiles/audio2.wav",
#     "audioFiles/audio3.wav"
# ]
# # Initialize this variable outside the function (e.g. at the top of the notebook)
# last_audio_time = 0
# AUDIO_COOLDOWN = 3.0  # seconds

# def update(change):
#     global blocked_slider, robot, last_audio_time
    
#     x = change['new'] 
#     x = preprocess(x)
#     y = model(x)
     
    
#     y = F.softmax(y, dim=1)
    
#     prob_blocked = float(y.flatten()[0])
#     #prob_blocked = 0.9
#     blocked_slider.value = prob_blocked
    
#     if prob_blocked < 0.5:
#         robot.forward(speed_slider.value)
#     else:
#         current_time = time.time()
#         if current_time - last_audio_time > AUDIO_COOLDOWN:
#             audio_file = random.choice(AUDIO_FILES)
#             display(Audio(filename=audio_file, autoplay=True))
#             last_audio_time = current_time
#         robot.left(speed_slider.value)
    
#     time.sleep(0.001)

# # Initialize once
# update({'new': camera.value})


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /media/nvidia/NVME/pytorch/pytorch-v1.9.0/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Cool! We've created our neural network execution function, but now we need to attach it to the camera for processing. 

We accomplish that with the ``observe`` function.

> WARNING: This code may move the robot!! Adjust the speed slider we defined earlier to control the base robot speed.  Some kits can move fast, so start slow, and gradually increase the value.

In [ ]:
camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera

Awesome! If your robot is plugged in it should now be generating new commands with each new camera frame.  Perhaps start by placing your robot on the ground and seeing what it does when it reaches an obstacle.

If you want to stop this behavior, you can unattach this callback by executing the code below.

In [ ]:
import time

camera.unobserve(update, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()

In [ ]:

robot.stop()

In [28]:
robot.stop()

In [29]:
camera.stop()

Perhaps you want the robot to run without streaming video to the browser.  You can unlink the camera as below.

In [ ]:
camera_link.unlink()  # don't stream to browser (will still run camera)

To continue streaming call the following.

In [ ]:
camera_link.link()  # stream to browser (wont run camera)

Again, let's close the camera conneciton properly so that we can use the camera in other notebooks.

In [ ]:
camera.stop()

### Conclusion

That's it for this live demo!  Hopefully you had some fun and your robot avoided collisions intelligently! 

If your robot wasn't avoiding collisions very well, try to spot where it fails.  The beauty is that we can collect more data for these failure scenarios
and the robot should get even better :)